In [21]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from sklearn import cross_validation, linear_model, ensemble, metrics

In [2]:
mu = 9.5
sigma = 0.4
n = 160
x_hat = 9.57
z_stat = (x_hat-mu) / (sigma/np.sqrt(n))
print z_stat

2.21359436212


In [31]:
p_value = scipy.stats.norm.cdf(-abs(z_stat))*2
print p_value

0.0268566955075


In [32]:
z = (9.57 - 9.5) / (0.4 / np.sqrt(160))
p = 2*(1-stats.norm.cdf(abs(z)))
print p

0.0268566955075


# Предсказание цены бриллианта

In [15]:
data = pd.read_csv('diamonds.txt', sep = '\t', header = 0)
data.head(5)

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


In [17]:
data.shape

(53940, 7)

In [19]:
ds_all = data.drop('price', 1)
ds_target = data.price

In [20]:
np.random.seed(1)
train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(ds_all, ds_target, test_size=0.25)

In [22]:
lin_reg = linear_model.LinearRegression()
lin_reg.fit(train_data, train_labels)
lin_reg_predict = lin_reg.predict(test_data)
x1 = np.abs(test_labels - lin_reg_predict)

print x1

2714      214.422356
14653    1016.460537
52760      41.878717
48658       7.165645
14812    1113.152701
37498     706.195885
12456     974.126765
16738     394.581497
21542     124.909872
40732     270.733881
29855     100.667975
44245     305.264158
45337    1314.234163
32171     493.323929
38297     242.868125
45810     637.110524
7896      398.295447
46466      68.195568
26651    1996.073257
21545    4167.051105
13263     531.454867
704       929.564338
12469    1670.744260
5825      890.286412
44479     507.322926
4218      679.365339
5083      554.727924
43189     123.216874
13003     513.540234
16692     349.855241
            ...     
5656       22.962535
36814     678.521367
12904     722.788004
30175     415.285025
30570     241.726337
45797      88.180006
2402      480.318374
6976       87.157646
7235       49.703161
19644    1331.411746
47880     420.844234
823       770.920387
14170     182.213151
50587     751.608640
11805     702.588791
20674    3542.390708
12651    1700

In [26]:
print metrics.mean_absolute_error(test_labels, lin_reg_predict)

890.376400429


In [24]:
rf = ensemble.RandomForestRegressor(random_state=1)
rf.fit(train_data, train_labels)
rf_predict = rf.predict(test_data)
x2 = np.abs(test_labels - rf_predict)

print x2

2714      294.550000
14653    1415.300000
52760     506.500000
48658      37.600000
14812    2375.500000
37498     232.000000
12456    1382.000000
16738     323.100000
21542     535.300000
40732     126.000000
29855     141.375000
44245      14.000000
45337     619.300000
32171     277.950000
38297      92.450000
45810     667.400000
7896      433.700000
46466      43.860000
26651     932.000000
21545    3934.300000
13263     247.100000
704       616.400000
12469    1338.500000
5825      837.100000
44479     114.100000
4218      508.700000
5083      396.000000
43189      21.600000
13003    1893.800000
16692     194.100000
            ...     
5656      209.900000
36814     249.335000
12904    1944.700000
30175     431.900000
30570       9.783333
45797      22.700000
2402      682.800000
6976      358.200000
7235      417.800000
19644    2488.500000
47880      18.200000
823       935.250000
14170     858.700000
50587     553.600000
11805    3811.700000
20674    2793.100000
12651    1827

In [ ]:
x1 = test_labels != predict1 и test_labels != predict2 

In [27]:
print metrics.mean_absolute_error(test_labels, rf_predict)

804.702882475


In [28]:
print stats.ttest_rel(x1, x2)

Ttest_relResult(statistic=12.745056775181958, pvalue=5.4286548181852529e-37)


In [30]:
print DescrStatsW(x1-x2).tconfint_mean()

(72.497265650112325, 98.849770256266368)


In [63]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [64]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [65]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [72]:
print "95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_ind(x1, x2)

95% confidence interval for a difference between proportions: [0.062533, 0.297467]


# Задача 3 - зевота

In [34]:
z = scipy.stats.norm.ppf(1-0.05/2)
f = 24
g = 4
n = 50
#_left = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
#_right = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
z_score = float(f - g) / np.sqrt(float(f+g) - (f-g)**2/n)
print scipy.stats.norm.cdf(-abs(z_score))*2

7.74421643104e-06


In [35]:
p1 = 10./34
p2 = 4./16
p = float(p1*34+p2*16)/50
z_score = float(p1-p2)/np.sqrt(p*(1-p)*(1./34+1./16))
print scipy.stats.norm.cdf(-abs(z_score))*2

0.74586091745


In [83]:
proportions_diff_z_test(proportions_diff_z_stat_ind([1]*4 + [0]*12, [1]*10 + [0]*24), 'less')

0.37293045872523534

# Задача про банкноты

In [36]:
data2 = pd.read_csv('banknotes.txt', sep = '\t', header = 0)
data2.head(5)

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [41]:
data2.columns = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'real']

In [43]:
ds2_all = data2.drop('real', 1)
ds2_target = data2.real

In [44]:
np.random.seed(1)
train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(ds2_all, ds2_target, test_size=50)

In [67]:
columns = ['X4', 'X5', 'X6']
train_data_X1 = train_data.drop(columns, axis=1)
test_data_X1 = test_data.drop(columns, axis=1)

columns = ['X1', 'X2', 'X3']
train_data_X4 = train_data.drop(columns, axis=1)
test_data_X4 = test_data.drop(columns, axis=1)


In [68]:
log_x1 = linear_model.LogisticRegression()
log_x1.fit(train_data_X1, train_labels)
x1_predict = log_x1.predict(test_data_X1)
x1 = np.abs(test_labels - x1_predict)

log_x2 = linear_model.LogisticRegression()
log_x2.fit(train_data_X4, train_labels)
x2_predict = log_x2.predict(test_data_X4)
x2 = np.abs(test_labels - x2_predict)


In [76]:
metrics.accuracy_score(test_labels, x1_predict)

0.80000000000000004

In [77]:
metrics.accuracy_score(test_labels, x2_predict)

0.97999999999999998

In [73]:
x1 = test_labels != x1_predict
x2 = test_labels != x2_predict

In [81]:
print proportions_diff_confint_rel(x1, x2)

(0.059945206279614305, 0.30005479372038568)


In [51]:
print np.sum(x1)

10


In [74]:
print np.mean(x1-x2)

0.22


C:\Users\gruyuri\Anaconda2\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '-' operator is not supported by numexpr for the bool dtype, use '^' instead
  unsupported[op_str]))


In [70]:
print stats.ttest_ind(x1, x2)

Ttest_indResult(statistic=2.973153822552955, pvalue=0.0037097805947425258)


In [75]:
print DescrStatsW(x1-x2).tconfint_mean()

(0.10107726963127962, 0.33892273036872039)


In [54]:
print DescrStatsW(x1-x2).tconfint_mean()

(0.055656626175543789, 0.3043433738244562)


# Задача про студентов - средний балл на экзамене

In [55]:
mu = 525.
sigma = 100.
n = 100
x_hat = 541.4
z_stat = (x_hat-mu) / (sigma/np.sqrt(n))
print z_stat

1.64


In [56]:
p = (1-stats.norm.cdf(abs(z_stat)))
print p

0.0505025834741


In [57]:
mu = 525.
sigma = 100.
n = 100
x_hat = 541.5
z_stat = (x_hat-mu) / (sigma/np.sqrt(n))
print z_stat

1.65


In [58]:
p = (1-stats.norm.cdf(abs(z_stat)))
print p

0.0494714680336


In [79]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)